In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
from tqdm.notebook import tqdm
import re

In [3]:
video_id = '9NYrsE_yW64'
subs = YouTubeTranscriptApi().list_transcripts(video_id)

In [4]:
print(subs)

For this video (9NYrsE_yW64) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-US ("Английский (Соединенные Штаты)")[TRANSLATABLE]

(GENERATED)
None

(TRANSLATION LANGUAGES)
 - az ("Азербайджанский")
 - sq ("Албанский")
 - am ("Амхарский")
 - en ("Английский")
 - ar ("Арабский")
 - hy ("Армянский")
 - af ("Африкаанс")
 - eu ("Баскский")
 - be ("Белорусский")
 - bn ("Бенгальский")
 - my ("Бирманский")
 - bg ("Болгарский")
 - bs ("Боснийский")
 - cy ("Валлийский")
 - hu ("Венгерский")
 - vi ("Вьетнамский")
 - haw ("Гавайский")
 - ht ("Гаитянский")
 - gl ("Галисийский")
 - el ("Греческий")
 - ka ("Грузинский")
 - gu ("Гуджарати")
 - gd ("Гэльский")
 - da ("Датский")
 - fy ("Западнофризский")
 - zu ("Зулу")
 - iw ("Иврит")
 - ig ("Игбо")
 - yi ("Идиш")
 - id ("Индонезийский")
 - ga ("Ирландский")
 - is ("Исландский")
 - es ("Испанский")
 - it ("Итальянский")
 - yo ("Йоруба")
 - kk ("Казахский")
 - kn ("Каннада")
 - ca ("Каталанский")
 - rw ("Киньяруанда")
 - k

In [4]:
subs

In [5]:
# Codes of languages I can understand.
OK_LANGUAGE_CODES = ['en-GB', 'en', 'en-US', 'ru']

for language_code in subs._manually_created_transcripts.keys():
    if language_code in OK_LANGUAGE_CODES:
        transcript = subs._manually_created_transcripts[
            language_code
        ].fetch()

In [9]:
transcript

[{'text': '- Most people are \noptimistic in the short-term',
  'start': 0.125,
  'duration': 2.085},
 {'text': 'and pessimistic in the long-term.',
  'start': 2.21,
  'duration': 2.211},
 {'text': "And what I've learned, \nis you actually need",
  'start': 4.421,
  'duration': 1.751},
 {'text': 'to be exactly the opposite of that.',
  'start': 6.172,
  'duration': 2.378},
 {'text': 'True founders need to be \npessimistic in the short-term,',
  'start': 8.55,
  'duration': 3.837},
 {'text': 'and long-term optimistic.', 'start': 12.387, 'duration': 2.419},
 {'text': "It's surprising, it's weird, \nbut let me explain why.",
  'start': 14.806,
  'duration': 3.795},
 {'text': "Let's get into it.", 'start': 18.601, 'duration': 0.835},
 {'text': '(bright upbeat music)', 'start': 19.436, 'duration': 3.169},
 {'text': 'One of the things I learned \nin therapy about myself',
  'start': 28.903,
  'duration': 3.17},
 {'text': 'is that I actually have an \nincredibly overactive super ego,',
  'sta

In [10]:
from pyyoutube import Api
with open('../../google-dev-token.txt') as f:
    API_KEY = f.read()
api = Api(api_key=API_KEY)

In [13]:
description = api.get_video_by_id(video_id=video_id).items[0].snippet.description
timestamps = re.findall('\d{1,2}:\d{2}', description)
timestamps

['00:00', '00:48', '02:43', '05:30']

In [8]:
def get_timestamps(video_id, video_subs, api=Api(api_key=API_KEY)):
    video_by_id = api.get_video_by_id(video_id=video_id)
    description = video_by_id.items[0].snippet.description
    timestamps = re.findall('\d{1,2}:\d{2}', description)
    start_seconds = []
    for timestamp in timestamps:
        mins, secs = timestamp.split(':')
        start_seconds.append(int(mins) * 60. + int(secs))
    start_seconds.append(video_subs[-1]['start'] + video_subs[-1]['duration'])
    if start_seconds[0] == 0.0:
        return start_seconds[1:]
    
    return start_seconds

start_seconds = get_timestamps(video_id, transcript, api)
start_seconds

[48.0, 163.0, 330.0, 503.836]

In [14]:
def convert_to_sentences(subs):
    script_sentences = []
    current_sentence = ''
    start_time = 0.

    for sub in subs:
        current_sentence += sub['text'].replace('\n', ' ') + ' '
        if sub['text'].endswith(('.', '!', '?')):
            script_sentences.append(
                {
                    'text': current_sentence,
                    'start': start_time,
                    'duration': round(sub['start'] + sub['duration'] - start_time, 2)
                }
            )
            current_sentence = ''
            start_time = round(sub['start'] + sub['duration'], 2)
    return script_sentences

In [15]:
script_sentences = convert_to_sentences(transcript)

In [16]:
script_sentences

[{'text': '- Most people are  optimistic in the short-term and pessimistic in the long-term. ',
  'start': 0.0,
  'duration': 4.42},
 {'text': "And what I've learned,  is you actually need to be exactly the opposite of that. ",
  'start': 4.42,
  'duration': 4.13},
 {'text': 'True founders need to be  pessimistic in the short-term, and long-term optimistic. ',
  'start': 8.55,
  'duration': 6.26},
 {'text': "It's surprising, it's weird,  but let me explain why. ",
  'start': 14.81,
  'duration': 3.79},
 {'text': "Let's get into it. ", 'start': 18.6, 'duration': 0.84},
 {'text': '(bright upbeat music) One of the things I learned  in therapy about myself is that I actually have an  incredibly overactive super ego, more so than most people. ',
  'start': 19.44,
  'duration': 17.97},
 {'text': "A super ego is the part of a person's mind that acts as a self critical conscience. ",
  'start': 37.41,
  'duration': 4.51},
 {'text': 'I grew up as a child  of Chinese immigrants. ',
  'start': 41

In [17]:
def generate_text(script_sentences, start_seconds):
    video_transcript = ''

    current_paragraph = 0

    for sub_time in script_sentences:
        if (
            current_paragraph < len(start_seconds) and
            sub_time['start'] + sub_time['duration'] > start_seconds[current_paragraph]
        ):
            if start_seconds[current_paragraph] > sub_time['start'] + 2.:
                video_transcript += sub_time['text']
                continue
            if len(video_transcript) > 10:
                current_paragraph += 1
                video_transcript += '\n\n'
        video_transcript += sub_time['text']
    generated_text = video_transcript.strip('_- ')
    generated_text = re.sub(r'\(.*?\) ', '', generated_text)
    return generated_text

In [18]:
print(generate_text(script_sentences, start_seconds))

Most people are  optimistic in the short-term and pessimistic in the long-term. And what I've learned,  is you actually need to be exactly the opposite of that. True founders need to be  pessimistic in the short-term, and long-term optimistic. It's surprising, it's weird,  but let me explain why. Let's get into it. One of the things I learned  in therapy about myself is that I actually have an  incredibly overactive super ego, more so than most people. A super ego is the part of a person's mind that acts as a self critical conscience. I grew up as a child  of Chinese immigrants. Really, anyone growing up  in a super strict household would have that same response. 

I have some good news for you,  if that's the case, though, this can be an asset. Here's Andy Grove, from his book, "Only The Paranoid Survive." He says, "I believe in  the value of paranoia. Business success contains the  seeds of its own destruction. The more successful you are, the more people want a  chunk of your busine

In [11]:
url = 'https://www.youtube.com/watch?v=9NYrsE_yW64'
url.split('=')[-1]

'9NYrsE_yW64'

In [19]:
import re

from typing import Any, Dict, List, Union

import pyyoutube
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._transcripts import TranscriptList


VideoSubs = List[Dict[str, Any]]


class YouTubeVideosHandler:

    # Codes of languages I can understand.
    OK_LANGUAGE_CODES = ['en-GB', 'en', 'en-US', 'ru']
    # Error message if no relevant subtitles are found.
    SORRY_MESSAGE = 'Unfortunately, there are no subs for this video :('
    
    def __init__(self, yt_api_client):
        self._yt_api_client = yt_api_client
        self._yt_transcripts_client = YouTubeTranscriptApi()
    
    @staticmethod
    def from_api_token_path(token_filepath: str):
        with open(token_filepath) as f:
            api_key = f.read()
        return YouTubeVideosHandler(
            yt_api_client=pyyoutube.Api(api_key=api_key),
        )
    
    def __call__(
            self,
            video_url_or_id: str,
            html_render=False,
    ) -> Dict[str, str]:
        video_id = video_url_or_id
        if 'youtube' in video_url_or_id:
            video_id = video_url_or_id.split('=')[-1]
        video_url = f'https://www.youtube.com/watch?v={video_id}'
        subs = self._yt_transcripts_client.list_transcripts(video_id)
        video_subs = self._get_video_subs(subs)
        if video_subs is None:
            return {
                'video_url': video_url,
                'video_id': video_id,
                'text': self.SORRY_MESSAGE,
            }
        start_seconds = self._get_timestamps(video_id, video_subs)
        script_sentences = self._convert_to_sentences(video_subs)
        text = self._generate_text(script_sentences, start_seconds)
        if html_render:
            text = text.replace('\n\n', '<br><br>')
        return {
            'video_url': video_url,
            'video_id': video_id,
            'text': text,
        }
    
    def _get_video_subs(
            self, subs: TranscriptList,
    ) -> Union[VideoSubs, None]:
        for language_code in subs._manually_created_transcripts.keys():
            if language_code in self.OK_LANGUAGE_CODES:
                return subs._manually_created_transcripts[
                    language_code
                ].fetch()
        return None
    
    def _get_timestamps(
            self,
            video_id: str,
            video_subs: VideoSubs,
    ) -> List[float]:
        video_by_id = self._yt_api_client.get_video_by_id(
            video_id=video_id,
        )
        description = video_by_id.items[0].snippet.description
        timestamps = re.findall('\d{1,2}:\d{2}', description)
        start_seconds = []

        for timestamp in timestamps:
            mins, secs = timestamp.split(':')
            start_seconds.append(int(mins) * 60. + int(secs))
        start_seconds.append(
            video_subs[-1]['start'] + video_subs[-1]['duration']
        )
        
        if start_seconds[0] == 0.0:
            return start_seconds[1:]
        return start_seconds
    
    def _convert_to_sentences(
            self, video_subs: VideoSubs,
    ) -> VideoSubs:
        script_sentences = []
        current_sentence = ''
        start_time = 0.

        for sub in video_subs:
            current_sentence += sub['text'].replace('\n', ' ') + ' '
            if sub['text'].endswith(('.', '!', '?')):
                duration = round(
                    sub['start'] + sub['duration'] - start_time, 2,
                )
                script_sentences.append(
                    {
                        'text': current_sentence,
                        'start': start_time,
                        'duration': duration,
                    }
                )
                current_sentence = ''
                start_time = duration
        return script_sentences
    
    def _generate_text(
            self,
            script_sentences: VideoSubs,
            start_seconds: List[float],
    ) -> str:
        video_transcript = ''
        current_paragraph = 0

        for sub_time in script_sentences:
            end_time = sub_time['start'] + sub_time['duration'] 
            if (
                current_paragraph < len(start_seconds) and
                end_time > start_seconds[current_paragraph]
            ):
                if start_seconds[current_paragraph] > sub_time['start'] + 2.:
                    video_transcript += sub_time['text']
                    continue
                if len(video_transcript) > 10:
                    current_paragraph += 1
                    video_transcript += '\n\n'
            video_transcript += sub_time['text']

        generated_text = video_transcript.strip('_- ')
        generated_text = re.sub(r'\(.*?\) ', '', generated_text)
        return generated_text

In [20]:
youtube_videos_handler = YouTubeVideosHandler.from_api_token_path('../../google-dev-token.txt')

In [24]:
video_id = '9NYrsE_yW64'
print(youtube_videos_handler(video_id, html_render=False))

{'video_url': 'https://www.youtube.com/watch?v=9NYrsE_yW64', 'video_id': '9NYrsE_yW64', 'text': 'Most people are  optimistic in the short-term and pessimistic in the long-term. And what I\'ve learned,  is you actually need to be exactly the opposite of that. True founders need to be  pessimistic in the short-term, and long-term optimistic. It\'s surprising, it\'s weird,  but let me explain why. Let\'s get into it. One of the things I learned  in therapy about myself is that I actually have an  incredibly overactive super ego, more so than most people. A super ego is the part of a person\'s mind that acts as a self critical conscience. I grew up as a child  of Chinese immigrants. Really, anyone growing up  in a super strict household would have that same response. I have some good news for you,  if that\'s the case, though, this can be an asset. Here\'s Andy Grove, from his book, "Only The Paranoid Survive." He says, "I believe in  the value of paranoia. Business success contains the  s